In [1]:
import numpy as np
import os
from pathlib import Path
import sklearn
import tensorflow as tf
from tensorflow import keras

In [2]:
def generate_time_series(batch_size, n_steps):
    freq1, freq2, offsets1, offsets2 = np.random.rand(4, batch_size, 1)
    time = np.linspace(0, 1, n_steps)
    
###넘파이 트릭으로 인해 50-10000은 반복연산되어 결국 10000의 크기를 가지게 된다
    series = 0.5 * np.sin((time - offsets1) * (freq1 * 10+10))
    series += 0.2 * np.sin((time - offsets2) * (freq2 * 20+20))
    series += 0.1 * (np.random.rand(batch_size, n_steps) - 0.5)
    return series[..., np.newaxis].astype(np.float32)

In [3]:
n_steps = 50
series = generate_time_series(10000, n_steps + 1)
X_train, y_train = series[:7000, :n_steps], series[:7000, -1]
X_valid, y_valid = series[7000:9000, :n_steps], series[7000:9000, -1]
X_test, y_test = series[9000:, :n_steps], series[9000:, -1]

In [4]:
### 51타임스텝의 시계열 데이터 생성 앞의 50개가 X마지막 1개가 y
X_train.shape, y_train.shape

((7000, 50, 1), (7000, 1))

In [5]:
y_pred = X_valid[:, -1]
np.mean(keras.losses.mean_squared_error(y_valid, y_pred))

0.02045599

In [6]:
model = keras.models.Sequential([
    keras.layers.Flatten(input_shape=[50, 1]),
    keras.layers.Dense(1)
])

model.compile(loss="mse", optimizer="adam")
history = model.fit(X_train, y_train, epochs=20,
                    validation_data=(X_valid, y_valid))

Train on 7000 samples, validate on 2000 samples
Epoch 1/20
7000/7000 [==============================] - 1s 106us/sample - loss: 0.3585 - val_loss: 0.1386
Epoch 2/20
7000/7000 [==============================] - 0s 55us/sample - loss: 0.0862 - val_loss: 0.0547
Epoch 3/20
7000/7000 [==============================] - 0s 51us/sample - loss: 0.0418 - val_loss: 0.0318
Epoch 4/20
7000/7000 [==============================] - 0s 55us/sample - loss: 0.0273 - val_loss: 0.0228
Epoch 5/20
7000/7000 [==============================] - 0s 55us/sample - loss: 0.0206 - val_loss: 0.0181
Epoch 6/20
7000/7000 [==============================] - 0s 52us/sample - loss: 0.0167 - val_loss: 0.0153
Epoch 7/20
7000/7000 [==============================] - 0s 52us/sample - loss: 0.0143 - val_loss: 0.0134
Epoch 8/20
7000/7000 [==============================] - 0s 53us/sample - loss: 0.0126 - val_loss: 0.0119
Epoch 9/20
7000/7000 [==============================] - 0s 53us/sample - loss: 0.0112 - val_loss: 0.0106
Epoch 

# RNN

In [7]:
model = keras.models.Sequential([
    keras.layers.SimpleRNN(1, input_shape = [None, 1])
])

optimizer = keras.optimizers.Adam(learning_rate=0.005)
model.compile(loss="mse", optimizer=optimizer)
history = model.fit(X_train, y_train, epochs=20,
                    validation_data=(X_valid, y_valid))

Train on 7000 samples, validate on 2000 samples
Epoch 1/20
7000/7000 [==============================] - 5s 714us/sample - loss: 0.0283 - val_loss: 0.0120
Epoch 2/20
7000/7000 [==============================] - 5s 645us/sample - loss: 0.0114 - val_loss: 0.0111
Epoch 3/20
7000/7000 [==============================] - 4s 604us/sample - loss: 0.0112 - val_loss: 0.0110
Epoch 4/20
7000/7000 [==============================] - 4s 606us/sample - loss: 0.0112 - val_loss: 0.0109
Epoch 5/20
7000/7000 [==============================] - 4s 606us/sample - loss: 0.0111 - val_loss: 0.0110
Epoch 6/20
7000/7000 [==============================] - 4s 603us/sample - loss: 0.0112 - val_loss: 0.0109
Epoch 7/20
7000/7000 [==============================] - 4s 599us/sample - loss: 0.0111 - val_loss: 0.0110
Epoch 8/20
7000/7000 [==============================] - 4s 602us/sample - loss: 0.0112 - val_loss: 0.0109
Epoch 9/20
7000/7000 [==============================] - 4s 597us/sample - loss: 0.0111 - val_loss: 0.011

# 심층RNN 

In [8]:
model = keras.models.Sequential([
    keras.layers.SimpleRNN(20, return_sequences=True, input_shape=[None, 1]),
    keras.layers.SimpleRNN(20, return_sequences=True),
    keras.layers.SimpleRNN(1)
])

model.compile(loss="mse", optimizer="adam")
history = model.fit(X_train, y_train, epochs=20,
                    validation_data=(X_valid, y_valid))

Train on 7000 samples, validate on 2000 samples
Epoch 1/20
7000/7000 [==============================] - 14s 2ms/sample - loss: 0.0198 - val_loss: 0.0045
Epoch 2/20
7000/7000 [==============================] - 12s 2ms/sample - loss: 0.0038 - val_loss: 0.0034
Epoch 3/20
7000/7000 [==============================] - 12s 2ms/sample - loss: 0.0033 - val_loss: 0.0033
Epoch 4/20
7000/7000 [==============================] - 12s 2ms/sample - loss: 0.0033 - val_loss: 0.0033
Epoch 5/20
7000/7000 [==============================] - 12s 2ms/sample - loss: 0.0032 - val_loss: 0.0031
Epoch 6/20
7000/7000 [==============================] - 13s 2ms/sample - loss: 0.0031 - val_loss: 0.0032
Epoch 7/20
7000/7000 [==============================] - 12s 2ms/sample - loss: 0.0030 - val_loss: 0.0030
Epoch 8/20
7000/7000 [==============================] - 12s 2ms/sample - loss: 0.0030 - val_loss: 0.0030
Epoch 9/20
7000/7000 [==============================] - 12s 2ms/sample - loss: 0.0029 - val_loss: 0.0028
Epoch 1

In [9]:
model = keras.models.Sequential([
    keras.layers.SimpleRNN(20, return_sequences=True, input_shape=[None, 1]),
    keras.layers.SimpleRNN(20),
    keras.layers.Dense(1)
])

model.compile(loss="mse", optimizer="adam")
history = model.fit(X_train, y_train, epochs=20,
                    validation_data=(X_valid, y_valid))

Train on 7000 samples, validate on 2000 samples
Epoch 1/20
7000/7000 [==============================] - 10s 1ms/sample - loss: 0.0134 - val_loss: 0.0043
Epoch 2/20
7000/7000 [==============================] - 8s 1ms/sample - loss: 0.0036 - val_loss: 0.0035
Epoch 3/20
7000/7000 [==============================] - 8s 1ms/sample - loss: 0.0031 - val_loss: 0.0033
Epoch 4/20
7000/7000 [==============================] - 8s 1ms/sample - loss: 0.0029 - val_loss: 0.0029
Epoch 5/20
7000/7000 [==============================] - 8s 1ms/sample - loss: 0.0029 - val_loss: 0.0030
Epoch 6/20
7000/7000 [==============================] - 8s 1ms/sample - loss: 0.0029 - val_loss: 0.0033
Epoch 7/20
7000/7000 [==============================] - 8s 1ms/sample - loss: 0.0029 - val_loss: 0.0033
Epoch 8/20
7000/7000 [==============================] - 8s 1ms/sample - loss: 0.0028 - val_loss: 0.0031
Epoch 9/20
7000/7000 [==============================] - 8s 1ms/sample - loss: 0.0029 - val_loss: 0.0028
Epoch 10/20
700

In [10]:
model = keras.models.Sequential([
    keras.layers.SimpleRNN(20, return_sequences=True, input_shape=[None, 1]),
    keras.layers.SimpleRNN(20),
    keras.layers.Dense(1, activation="relu")
])

model.compile(loss="mse", optimizer="adam")
history = model.fit(X_train, y_train, epochs=20,
                    validation_data=(X_valid, y_valid))

Train on 7000 samples, validate on 2000 samples
Epoch 1/20
7000/7000 [==============================] - 10s 1ms/sample - loss: 0.0916 - val_loss: 0.0689
Epoch 2/20
7000/7000 [==============================] - 8s 1ms/sample - loss: 0.0685 - val_loss: 0.0662
Epoch 3/20
7000/7000 [==============================] - 8s 1ms/sample - loss: 0.0676 - val_loss: 0.0658
Epoch 4/20
7000/7000 [==============================] - 8s 1ms/sample - loss: 0.0675 - val_loss: 0.0657
Epoch 5/20
7000/7000 [==============================] - 8s 1ms/sample - loss: 0.0674 - val_loss: 0.0657
Epoch 6/20
7000/7000 [==============================] - 8s 1ms/sample - loss: 0.0674 - val_loss: 0.0656
Epoch 7/20
7000/7000 [==============================] - 8s 1ms/sample - loss: 0.0674 - val_loss: 0.0657
Epoch 8/20
7000/7000 [==============================] - 8s 1ms/sample - loss: 0.0674 - val_loss: 0.0655
Epoch 9/20
7000/7000 [==============================] - 8s 1ms/sample - loss: 0.0674 - val_loss: 0.0655
Epoch 10/20
700

### 타임스텝의 예측값을 순차적으로 더해 다음 값을 예측하는 경우

In [11]:
series = generate_time_series(1, n_steps + 10)
X_new, Y_new = series[:, :n_steps], series[:, n_steps:]
X = X_new

#predict한걸 X에 더해가며 계속해서 다음 값 예측
for step_ahead in range(10):
    y_pred_one = model.predict(X[:, step_ahead:])[:, np.newaxis, :]
    X = np.concatenate([X, y_pred_one], axis=1)
    
Y_pred = X[:, n_steps:]

In [12]:
Y_pred.shape

(1, 10, 1)

In [13]:
np.mean(keras.metrics.mean_squared_error(Y_new, Y_pred))

0.038206827

### 일정량의 타임스텝을 한번에 예측하는 경우

In [14]:
n_steps = 50
series = generate_time_series(10000, n_steps + 10)
X_train, Y_train = series[:7000, :n_steps], series[:7000, -10:, 0]
X_valid, Y_valid = series[7000:9000, :n_steps], series[7000:9000, -10:, 0]
X_test, Y_test = series[9000:, :n_steps], series[9000:, -10:, 0]

In [15]:
model = keras.models.Sequential([
    keras.layers.SimpleRNN(20, return_sequences=True, input_shape=[None, 1]),
    keras.layers.SimpleRNN(20),
    keras.layers.Dense(10)
])

model.compile(loss="mse", optimizer="adam")
history = model.fit(X_train, Y_train, epochs=20,
                    validation_data=(X_valid, Y_valid))

Train on 7000 samples, validate on 2000 samples
Epoch 1/20
7000/7000 [==============================] - 9s 1ms/sample - loss: 0.0583 - val_loss: 0.0305
Epoch 2/20
7000/7000 [==============================] - 8s 1ms/sample - loss: 0.0263 - val_loss: 0.0213
Epoch 3/20
7000/7000 [==============================] - 8s 1ms/sample - loss: 0.0191 - val_loss: 0.0194
Epoch 4/20
7000/7000 [==============================] - 8s 1ms/sample - loss: 0.0162 - val_loss: 0.0141
Epoch 5/20
7000/7000 [==============================] - 8s 1ms/sample - loss: 0.0144 - val_loss: 0.0155
Epoch 6/20
7000/7000 [==============================] - 8s 1ms/sample - loss: 0.0136 - val_loss: 0.0144
Epoch 7/20
7000/7000 [==============================] - 8s 1ms/sample - loss: 0.0134 - val_loss: 0.0117
Epoch 8/20
7000/7000 [==============================] - 8s 1ms/sample - loss: 0.0129 - val_loss: 0.0129
Epoch 9/20
7000/7000 [==============================] - 8s 1ms/sample - loss: 0.0118 - val_loss: 0.0112
Epoch 10/20
7000

In [16]:
Y = np.empty((10000, n_steps, 10))
for step_ahead in range(1, 10 + 1):
    Y[:, :,step_ahead -1] = series[:, step_ahead:step_ahead + n_steps, 0]    
##### 결과적으로 y의 첫번쨰 값은 1~51의 샘플을가지고 이걸 10번 반복한다 이런 뜻?

y_train = Y[:7000]
y_valid = Y[7000:9000]
y_test = Y[9000:]

In [17]:
X_train.shape, y_train.shape

((7000, 50, 1), (7000, 50, 10))

In [18]:
model = keras.models.Sequential([
    keras.layers.SimpleRNN(20, return_sequences=True, input_shape=[None, 1]),
    keras.layers.SimpleRNN(20, return_sequences=True),
    keras.layers.TimeDistributed(keras.layers.Dense(10))
])

In [19]:
def last_time_step_mse(Y_true, Y_pred):
    return keras.metrics.mean_squared_error(Y_true[:, -1], Y_pred[:, -1])

optimizer = keras.optimizers.Adam(lr=0.01)
model.compile(loss="mse", optimizer = optimizer, metrics=[last_time_step_mse])
history = model.fit(X_train, y_train, epochs=20,validation_data=(X_valid, y_valid))

Train on 7000 samples, validate on 2000 samples
Epoch 1/20
7000/7000 [==============================] - 10s 1ms/sample - loss: 0.0503 - last_time_step_mse: 0.0389 - val_loss: 0.0393 - val_last_time_step_mse: 0.0270
Epoch 2/20
7000/7000 [==============================] - 8s 1ms/sample - loss: 0.0355 - last_time_step_mse: 0.0229 - val_loss: 0.0313 - val_last_time_step_mse: 0.0171
Epoch 3/20
7000/7000 [==============================] - 8s 1ms/sample - loss: 0.0318 - last_time_step_mse: 0.0192 - val_loss: 0.0301 - val_last_time_step_mse: 0.0191
Epoch 4/20
7000/7000 [==============================] - 8s 1ms/sample - loss: 0.0288 - last_time_step_mse: 0.0156 - val_loss: 0.0283 - val_last_time_step_mse: 0.0155
Epoch 5/20
7000/7000 [==============================] - 8s 1ms/sample - loss: 0.0260 - last_time_step_mse: 0.0123 - val_loss: 0.0250 - val_last_time_step_mse: 0.0106
Epoch 6/20
7000/7000 [==============================] - 8s 1ms/sample - loss: 0.0239 - last_time_step_mse: 0.0111 - val_l

In [20]:
from tensorflow.keras.layers import LayerNormalization

class LNSimpleRNNCell(keras.layers.Layer):
    ##### 이 부분만  수정하여 normalization, 활성화 함수, drop out여부 조정 가능
    def __init__(self, units, activation="tanh", **kwargs):
        super().__init__(**kwargs)
        self.state_size = units
        self.output_size = units
        self.simple_rnn_cell = keras.layers.SimpleRNNCell(units,
                                                          activation=None)
        self.layer_norm = LayerNormalization()
        self.activation = keras.activations.get(activation)
    def get_initial_state(self, inputs=None, batch_size=None, dtype=None):
        if inputs is not None:
            batch_size = tf.shape(inputs)[0]
            dtype = inputs.dtype
        return [tf.zeros([batch_size, self.state_size], dtype=dtype)]
    def call(self, inputs, states):
        outputs, new_states = self.simple_rnn_cell(inputs, states)
        norm_outputs = self.activation(self.layer_norm(outputs))
        return norm_outputs, [norm_outputs]

In [21]:
model = keras.models.Sequential([
    keras.layers.RNN(LNSimpleRNNCell(20), return_sequences=True,
                     input_shape=[None, 1]),
    keras.layers.RNN(LNSimpleRNNCell(20), return_sequences=True),
    keras.layers.TimeDistributed(keras.layers.Dense(10))
])

model.compile(loss="mse", optimizer="adam", metrics=[last_time_step_mse])
history = model.fit(X_train, y_train, epochs=20,
                    validation_data=(X_valid, y_valid))

Train on 7000 samples, validate on 2000 samples
Epoch 1/20
7000/7000 [==============================] - 17s 2ms/sample - loss: 0.1606 - last_time_step_mse: 0.1490 - val_loss: 0.0731 - val_last_time_step_mse: 0.0662
Epoch 2/20
7000/7000 [==============================] - 15s 2ms/sample - loss: 0.0635 - last_time_step_mse: 0.0571 - val_loss: 0.0561 - val_last_time_step_mse: 0.0496
Epoch 3/20
7000/7000 [==============================] - 15s 2ms/sample - loss: 0.0522 - last_time_step_mse: 0.0403 - val_loss: 0.0478 - val_last_time_step_mse: 0.0352
Epoch 4/20
7000/7000 [==============================] - 15s 2ms/sample - loss: 0.0460 - last_time_step_mse: 0.0323 - val_loss: 0.0430 - val_last_time_step_mse: 0.0296
Epoch 5/20
7000/7000 [==============================] - 15s 2ms/sample - loss: 0.0418 - last_time_step_mse: 0.0284 - val_loss: 0.0394 - val_last_time_step_mse: 0.0257
Epoch 6/20
7000/7000 [==============================] - 15s 2ms/sample - loss: 0.0386 - last_time_step_mse: 0.0254 - 

# LSTM 

In [22]:
model = keras.models.Sequential([
    keras.layers.LSTM(20, return_sequences=True, input_shape=[None,1]),
    keras.layers.LSTM(20, return_sequences=True),
    keras.layers.TimeDistributed(keras.layers.Dense(10))
])

model.compile(loss="mse", optimizer="adam", metrics=[last_time_step_mse])
history = model.fit(X_train, y_train, epochs=20,
                    validation_data=(X_valid, y_valid))

Train on 7000 samples, validate on 2000 samples
Epoch 1/20
7000/7000 [==============================] - 5s 682us/sample - loss: 0.0766 - last_time_step_mse: 0.0610 - val_loss: 0.0539 - val_last_time_step_mse: 0.0337
Epoch 2/20
7000/7000 [==============================] - 2s 282us/sample - loss: 0.0466 - last_time_step_mse: 0.0262 - val_loss: 0.0421 - val_last_time_step_mse: 0.0230
Epoch 3/20
7000/7000 [==============================] - 2s 278us/sample - loss: 0.0384 - last_time_step_mse: 0.0176 - val_loss: 0.0362 - val_last_time_step_mse: 0.0151
Epoch 4/20
7000/7000 [==============================] - 2s 281us/sample - loss: 0.0344 - last_time_step_mse: 0.0148 - val_loss: 0.0331 - val_last_time_step_mse: 0.0137
Epoch 5/20
7000/7000 [==============================] - 2s 282us/sample - loss: 0.0320 - last_time_step_mse: 0.0132 - val_loss: 0.0314 - val_last_time_step_mse: 0.0136
Epoch 6/20
7000/7000 [==============================] - 2s 283us/sample - loss: 0.0305 - last_time_step_mse: 0.0

# GRU셀 

In [23]:
model = keras.models.Sequential([
    keras.layers.GRU(20, return_sequences=True, input_shape=[None, 1]),
    keras.layers.GRU(20, return_sequences=True),
    keras.layers.TimeDistributed(keras.layers.Dense(10))
])

model.compile(loss="mse", optimizer="adam", metrics=[last_time_step_mse])
history = model.fit(X_train, y_train, epochs=20,
                    validation_data=(X_valid, y_valid))

Train on 7000 samples, validate on 2000 samples
Epoch 1/20
7000/7000 [==============================] - 5s 652us/sample - loss: 0.0746 - last_time_step_mse: 0.0662 - val_loss: 0.0528 - val_last_time_step_mse: 0.0433
Epoch 2/20
7000/7000 [==============================] - 2s 274us/sample - loss: 0.0485 - last_time_step_mse: 0.0378 - val_loss: 0.0458 - val_last_time_step_mse: 0.0353
Epoch 3/20
7000/7000 [==============================] - 2s 274us/sample - loss: 0.0443 - last_time_step_mse: 0.0334 - val_loss: 0.0420 - val_last_time_step_mse: 0.0308
Epoch 4/20
7000/7000 [==============================] - 2s 272us/sample - loss: 0.0403 - last_time_step_mse: 0.0289 - val_loss: 0.0380 - val_last_time_step_mse: 0.0260
Epoch 5/20
7000/7000 [==============================] - 2s 272us/sample - loss: 0.0362 - last_time_step_mse: 0.0238 - val_loss: 0.0369 - val_last_time_step_mse: 0.0265
Epoch 6/20
7000/7000 [==============================] - 2s 269us/sample - loss: 0.0328 - last_time_step_mse: 0.0

In [24]:
model = keras.models.Sequential([
    keras.layers.Conv1D(filters=20, kernel_size=4, strides=2, padding="valid",
                        input_shape=[None, 1]),
    keras.layers.GRU(20, return_sequences=True),
    keras.layers.GRU(20, return_sequences=True),
    keras.layers.TimeDistributed(keras.layers.Dense(10))
])

model.compile(loss="mse", optimizer="adam", metrics=[last_time_step_mse])
history = model.fit(X_train, y_train[:, 3::2], epochs=20,
                    validation_data=(X_valid, y_valid[:, 3::2]))

Train on 7000 samples, validate on 2000 samples
Epoch 1/20
7000/7000 [==============================] - 5s 646us/sample - loss: 0.0698 - last_time_step_mse: 0.0615 - val_loss: 0.0460 - val_last_time_step_mse: 0.0377
Epoch 2/20
7000/7000 [==============================] - 2s 247us/sample - loss: 0.0391 - last_time_step_mse: 0.0311 - val_loss: 0.0351 - val_last_time_step_mse: 0.0275
Epoch 3/20
7000/7000 [==============================] - 2s 243us/sample - loss: 0.0327 - last_time_step_mse: 0.0239 - val_loss: 0.0308 - val_last_time_step_mse: 0.0214
Epoch 4/20
7000/7000 [==============================] - 2s 245us/sample - loss: 0.0292 - last_time_step_mse: 0.0193 - val_loss: 0.0283 - val_last_time_step_mse: 0.0177
Epoch 5/20
7000/7000 [==============================] - 2s 244us/sample - loss: 0.0269 - last_time_step_mse: 0.0164 - val_loss: 0.0265 - val_last_time_step_mse: 0.0156
Epoch 6/20
7000/7000 [==============================] - 2s 245us/sample - loss: 0.0253 - last_time_step_mse: 0.0